In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
KEY  = os.environ['JINA_EMBEDDING_KEY']


In [2]:
! pip freeze > requirements.txt

In [2]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("documents/temp.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [4]:
splits = text_splitter.split_documents(docs)
len(splits)

32

In [5]:
from langchain.embeddings import JinaEmbeddings
embedding = JinaEmbeddings(jina_api_key = KEY, model_name="jina-embeddings-v2-base-en")


In [9]:
from langchain.vectorstores import Chroma
# persist_directory = 'docs/chroma1/'
persist_directory = 'chroma4/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [8]:
! pip install chromadb

  Using cached build-1.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached chroma_hnswlib-0.7.3-cp311-cp311-win_amd64.whl.metadata (262 bytes)
  Using cached pulsar_client-3.4.0-cp311-cp311-win_amd64.whl.metadata (1.0 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.1.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached bcrypt-4.1.2-cp39-abi3-win_amd64.whl.metadata (9.8 kB)
  Using cached kubernetes-29.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached mmh3-4.1.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached websocket_client-1.7.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cach

In [10]:
print(vectordb._collection.count())

32


In [11]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [12]:
vectordb.persist()

In [13]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [14]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

c:\Users\DELL\.conda\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
from langchain.chains import RetrievalQA

In [14]:
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever()
# )
# from langchain.prompts import PromptTemplate


# question="how many words are there?"
# result = qa_chain({"query": question})
# result['result']

In [15]:
# from langchain.prompts import PromptTemplate

# # Build prompt
# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
# {context}
# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [16]:
# # Run chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm,
#     retriever=vectordb.as_retriever(),
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
# )

In [16]:

# Build prompt
from langchain.prompts import PromptTemplate

# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "Fuck you !" at the end of the answer. 
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Must say "Fuck You!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Run chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]


# ### Memory

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# ### ConversationalRetrievalChain


from langchain.chains import ConversationalRetrievalChain

retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)


c:\Users\DELL\.conda\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [17]:
chat_history = []
answer = ""
db_query = ""
db_response = []

In [19]:
query ="what is this document?"

result = qa({"question": query, "chat_history": chat_history})
chat_history.extend([(query, result["answer"])])
# db_query = result["generated_question"]
# db_response = result["source_documents"]
answer = result["answer"]
print(answer)

This document is a data sheet for the LM741 operational amplifier provided by Texas Instruments. It includes information about device and documentation support, community resources, trademarks, electrostatic discharge caution, glossary, mechanical, packaging, and orderable information. It also provides details about RoHS compliance, lead finish options, and other technical specifications related to the LM741 operational amplifier.
